In [9]:
WOS_CREDENTIALS = {
    "url": "https://zen-cpd-zen.cpd-outcomes-00-57d48117af6edd64ea27fa1142b91b0b-0000.us-south.containers.appdomain.cloud",
    "username": "admin",
    "password": "********",
    "version": "3.5"
}

In [10]:
!pip install ibm-cloud-sdk-core --no-cache | tail -n 1
!pip install --upgrade ibm-watson-machine-learning --user | tail -n 1
!pip install --upgrade ibm-watson-openscale --no-cache | tail -n 1

In [11]:
import os
token = os.environ['USER_ACCESS_TOKEN']

WML_CREDENTIALS = {
   "token" : token,
   "instance_id" : "wml_local",
   "url": WOS_CREDENTIALS["url"],
   "version": "3.5"
}

In [12]:
from ibm_watson_openscale import APIClient
from ibm_watson_openscale.utils import *
from ibm_watson_openscale.supporting_classes import *
from ibm_watson_openscale.supporting_classes.enums import *

import json
import requests
import base64
from requests.auth import HTTPBasicAuth
import time

In [27]:
DATAMART_ID = '00000000-0000-0000-0000-1616425950709603'

In [18]:
authenticator = CloudPakForDataAuthenticator(
        url=WOS_CREDENTIALS['url'],
        username=WOS_CREDENTIALS['username'],
        password=WOS_CREDENTIALS['password'],
        disable_ssl_verification=True
    )

wos_client = APIClient(service_url=WOS_CREDENTIALS['url'], service_instance_id=DATAMART_ID, authenticator=authenticator)
wos_client.version

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'zen-cpd-zen.cpd-outcomes-00-57d48117af6edd64ea27fa1142b91b0b-0000.us-south.containers.appdomain.cloud'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'zen-cpd-zen.cpd-outcomes-00-57d48117af6edd64ea27fa1142b91b0b-0000.us-south.containers.appdomain.cloud'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


'3.0.1'

In [19]:
wos_client.data_marts.show()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'zen-cpd-zen.cpd-outcomes-00-57d48117af6edd64ea27fa1142b91b0b-0000.us-south.containers.appdomain.cloud'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [20]:
wos_client.service_providers.show()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'zen-cpd-zen.cpd-outcomes-00-57d48117af6edd64ea27fa1142b91b0b-0000.us-south.containers.appdomain.cloud'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [21]:
import json
from ibm_watson_machine_learning import APIClient

wml_client = APIClient(WML_CREDENTIALS)
wml_client.version

'1.0.66'

In [22]:
wml_client.spaces.list()

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  -----------------------------------------------------------------------  ------------------------
ID                                    NAME                                                                     CREATED
21844281-0655-4de1-af12-700ed36a5d1e  attrition-production                                                     2021-03-29T18:10:56.948Z
20213e0d-102b-4113-a317-8a5c2d8f2c01  openscale-express-path-preprod-00000000-0000-0000-0000-1616425950709603  2021-03-22T15:21:26.721Z
93d7666e-7df9-4cbb-af65-6eb93838e644  openscale-express-path-00000000-0000-0000-0000-1616425950709603          2021-03-22T15:21:09.146Z
------------------------------------  -----------------------------------------------------------------------  ------------------------


In [23]:
WML_SPACE_ID = '21844281-0655-4de1-af12-700ed36a5d1e'

In [24]:
SERVICE_PROVIDER_NAME = "Attrition Production Space"
SERVICE_PROVIDER_DESCRIPTION = "Production space for the HR attrition model"

In [25]:
service_providers = wos_client.service_providers.list().result.service_providers
for service_provider in service_providers:
    service_instance_name = service_provider.entity.name
    if service_instance_name == SERVICE_PROVIDER_NAME:
        service_provider_id = service_provider.metadata.id
        wos_client.service_providers.delete(service_provider_id)
        print("Deleted existing service_provider for WML instance: {}".format(service_provider_id))

In [26]:
added_service_provider_result = wos_client.service_providers.add(
        name=SERVICE_PROVIDER_NAME,
        description=SERVICE_PROVIDER_DESCRIPTION,
        service_type=ServiceTypes.WATSON_MACHINE_LEARNING,
        deployment_space_id = WML_SPACE_ID,
        operational_space_id = "production",
        credentials=WML_CREDENTIALS,
        background_mode=False
    ).result
service_provider_id = added_service_provider_result.metadata.id




 Waiting for end of adding service provider 15738fbb-1260-4767-8975-b22eb15c0ce9 




active

-----------------------------------------------
 Successfully finished adding service provider 
-----------------------------------------------




In [31]:
wml_client.set.default_space(WML_SPACE_ID)

'SUCCESS'

In [32]:
wml_client.deployments.list()

------------------------------------  ---------------------------  -----  ------------------------
GUID                                  NAME                         STATE  CREATED
6b8d6961-16d2-4c1d-9389-be2c300e8bb7  predictive attrition - 2018  ready  2021-03-29T18:12:34.965Z
------------------------------------  ---------------------------  -----  ------------------------


In [34]:
DEPLOYMENT_UID = '6b8d6961-16d2-4c1d-9389-be2c300e8bb7'

In [37]:
asset_deployment_details = wos_client.service_providers.list_assets(data_mart_id=DATAMART_ID, service_provider_id=service_provider_id, deployment_id=DEPLOYMENT_UID, deployment_space_id = WML_SPACE_ID).result['resources'][0]
asset_deployment_details

{'metadata': {'guid': '6b8d6961-16d2-4c1d-9389-be2c300e8bb7',
  'created_at': '2021-03-29T18:12:34.965Z',
  'modified_at': '2021-03-29T18:12:34.965Z'},
 'entity': {'name': 'predictive attrition - 2018',
  'type': 'online',
  'scoring_endpoint': {'url': 'https://ibm-nginx-svc.zen.svc.cluster.local/ml/v4/deployments/6b8d6961-16d2-4c1d-9389-be2c300e8bb7/predictions'},
  'asset': {},
  'asset_properties': {}}}

In [40]:
model_asset_details_from_deployment=wos_client.service_providers.get_deployment_asset(data_mart_id=DATAMART_ID,service_provider_id=service_provider_id,deployment_id=DEPLOYMENT_UID,deployment_space_id=WML_SPACE_ID)

## Remove existing subscription here if dropping service provider isn't enough

In [41]:
import pandas as pd

df_data_1 = pd.read_csv('/project_data/data_asset/autoai_records_2020.csv')
df_data_1.head()

POSITION_CODE  DEPARTMENT_CODE  DAYS_WITH_COMPANY  COMMUTE_TIME  \
0           5700              390                940            64   
1           3800              300               1730             6   
2           5400              390               1573            12   
3           2000              320               1640            15   
4           4500              360               2065            30   

   AGE_BEGIN_PERIOD  GENDER_CODE  ATTRITION  PERIOD_TOTAL_DAYS  \
0                48            0          0                330   
1                37            1          0                330   
2                40            0          0                330   
3                45            1          0                330   
4                49            0          0                330   

   STARTING_SALARY  ENDING_SALARY  ...  VACATION_DAYS_TAKEN  SICK_DAYS_TAKEN  \
0         52230.77       54961.54  ...                   10              7.5   
1         40384.62       42846.15  ...                   11              7.5   
2         67653.85       67653.85  ...                   15             11.0   
3         60000.00       63500.00  ...                   15             10.0   
4        109230.77      116538.46  ...                   15              9.0   

   PROMOTIONS  NB_MANAGERS  DAYS_IN_POSITION  DAYS_SINCE_LAST_RAISE  \
0           0            1               940                    210   
1           0            1              1730                    270   
2           0            1              1060                    330   
3           0            1              1640                    180   
4           0            1              2065                    240   

   RANKING_CODE  OVERTIME  DBLOVERTIME  TRAVEL  
0             4       0.0          0.0       0  
1             4       0.0          0.0       0  
2             4       0.0          0.0       0  
3             2       0.0          0.0       0  
4             2       0.0          0.0       0  

[5 rows x 23 columns]

In [44]:
feature_columns = df_data_1.drop(['ATTRITION'], axis=1).columns.tolist()
cat_features = []

In [46]:
subscription_details = wos_client.subscriptions.add(
        data_mart_id=DATAMART_ID,
        service_provider_id=service_provider_id,
        asset=Asset(
            asset_id=model_asset_details_from_deployment["entity"]["asset"]["asset_id"],
            name=model_asset_details_from_deployment["entity"]["asset"]["name"],
            url=model_asset_details_from_deployment["entity"]["asset"]["url"],
            asset_type=AssetTypes.MODEL,
            input_data_type=InputDataType.STRUCTURED,
            problem_type=ProblemType.BINARY_CLASSIFICATION
        ),
        deployment=AssetDeploymentRequest(
            deployment_id=asset_deployment_details['metadata']['guid'],
            name=asset_deployment_details['entity']['name'],
            deployment_type= DeploymentTypes.ONLINE,
            url=asset_deployment_details['entity']['scoring_endpoint']['url']
        ),
        asset_properties=AssetPropertiesRequest(
            label_column="ATTRITION",
            probability_fields=["probability"],
            prediction_field="prediction",
            feature_fields = feature_columns,
            categorical_fields = cat_features,
            training_data_reference=TrainingDataReference(type="cos",
                                                          location=COSTrainingDataReferenceLocation(bucket = "faststartlab-donotdelete-pr-nhfd4jnhlxgpc7",
                                                                                                    file_name = "data_2019.csv"),
                                                          connection=COSTrainingDataReferenceConnection.from_dict({
                                                                        "resource_instance_id": "crn:v1:bluemix:public:cloud-object-storage:global:a/7d8b3c34272c0980d973d3e40be9e9d2:2883ef10-23f1-4592-8582-2f2ef4973639::",
                                                                        "url": "https://s3.us.cloud-object-storage.appdomain.cloud",
                                                                        "api_key": "yqcPbWZ0AQPHleHVerrR4Wx5e9pymBdMgydbEra5zCif",
                                                                        "iam_url": "https://iam.ng.bluemix.net/oidc/token"}))
        )
    ).result
subscription_id = subscription_details.metadata.id
print('subscription_id: ' + subscription_id)

subscription_id: 8c38a9ae-dff7-4582-b363-d8194d00bc71


In [47]:
import time

time.sleep(5)
payload_data_set_id = None
payload_data_set_id = wos_client.data_sets.list(type=DataSetTypes.PAYLOAD_LOGGING, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id
if payload_data_set_id is None:
    print("Payload data set not found. Please check subscription status.")
else:
    print("Payload data set id:", payload_data_set_id)

Payload data set id: b63b2013-ebe1-4d76-86c3-92201c4c67d4


In [48]:
feed_data = df_data_1.drop(['ATTRITION'], axis=1).sample(n=10)

In [50]:
fields = feed_data.columns.tolist()
values = feed_data.values.tolist()

payload_scoring = {"input_data": [{"fields": fields, "values": values}]}

In [52]:
predictions = wml_client.deployments.score(DEPLOYMENT_UID, payload_scoring)
predictions

{'predictions': [{'fields': ['prediction', 'probability'],
   'values': [[0, [0.7029790457048303, 0.2970209542951697]],
    [0, [0.9477069695883402, 0.05229303041165983]],
    [0, [0.9990554239516338, 0.0009445760483662239]],
    [1, [0.15442770510789627, 0.8455722948921037]],
    [0, [0.9781882796374143, 0.021811720362585756]],
    [1, [0.11770849456375698, 0.882291505436243]],
    [1, [0.05566454175135693, 0.9443354582486431]],
    [0, [0.9987508945494241, 0.001249105450575938]],
    [1, [0.08729752455748419, 0.9127024754425158]],
    [0, [0.9860694212932352, 0.013930578706764892]]]}]}

In [53]:
time.sleep(5)
pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
print("Number of records in the payload logging table: {}".format(pl_records_count))
if pl_records_count == 0:
    raise Exception("Payload logging did not happen!")

Number of records in the payload logging table: 10


In [55]:
monitor_instances = wos_client.monitor_instances.list().result.monitor_instances
for monitor_instance in monitor_instances:
    monitor_def_id=monitor_instance.entity.monitor_definition_id
    if monitor_def_id == "drift" and monitor_instance.entity.target.target_id == subscription_id:
        wos_client.monitor_instances.delete(monitor_instance.metadata.id)
        print('Deleted existing drift monitor instance with id: ', monitor_instance.metadata.id)


target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id

)

parameters = {
    "min_samples": 100,
    "drift_threshold": 0.05,
    "train_drift_model": True,
    "enable_model_drift": True,
    "enable_data_drift": True
}

drift_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=DATAMART_ID,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.DRIFT.ID,
    target=target,
    parameters=parameters
).result

drift_monitor_instance_id = drift_monitor_details.metadata.id
drift_monitor_instance_id




 Waiting for end of monitor instance creation 2a68b872-edb4-4505-8730-73a23e1cf93d 




preparing.....
active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




'2a68b872-edb4-4505-8730-73a23e1cf93d'

In [56]:
target = Target(
        target_type=TargetTypes.SUBSCRIPTION,
        target_id=subscription_id
)
parameters = {
    "min_feedback_data_size": 50
}
thresholds = [
    {
        "metric_id": "area_under_roc",
        "type": "lower_limit",
        "value": 0.8
    }
]
quality_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=DATAMART_ID,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.QUALITY.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds 
).result
quality_monitor_instance_id = quality_monitor_details.metadata.id
quality_monitor_instance_id




 Waiting for end of monitor instance creation e673aed6-0d90-48f2-af64-d58a1a04cd70 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




'e673aed6-0d90-48f2-af64-d58a1a04cd70'

In [57]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)
parameters = {
    "enabled": True
}
explainability_details = wos_client.monitor_instances.create(
    data_mart_id=DATAMART_ID,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.EXPLAINABILITY.ID,
    target=target,
    parameters=parameters
).result

explainability_monitor_id = explainability_details.metadata.id




 Waiting for end of monitor instance creation f993544a-115e-4e58-923d-954f4ce6a88d 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------


